In [1]:
import torch
import syft as sy
hook = sy.TorchHook()
me = hook.local_worker

In [2]:
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)
remote = sy.VirtualWorker(id="remote",hook=hook, is_client_worker=False)
me.is_client_worker = False
bob.add_workers([me, alice, remote])
alice.add_workers([me, bob, remote])
remote.add_workers([me, bob, alice])

In [3]:
x = torch.FloatTensor([1, 2, -3, 4, 5]).send(remote)
print('**** Objects')
print(me._objects)
print(remote._objects)
y = x.cos()
print('**** Objects')
print(me._objects)
print(remote._objects)
y.get()

**** Objects
{3652176125: <_PointerTensor - id:3652176125 owner:0 loc:remote id@loc:1531110264>}
{1531110264: <_LocalTensor - id:1531110264 owner:remote>}
**** Objects
{3652176125: <_PointerTensor - id:3652176125 owner:0 loc:remote id@loc:1531110264>, 6187787624: <_PointerTensor - id:6187787624 owner:0 loc:remote id@loc:4911491334>}
{1531110264: <_LocalTensor - id:1531110264 owner:remote>, 4911491334: <_LocalTensor - id:4911491334 owner:remote>}



 0.5403
-0.4161
-0.9900
-0.6536
 0.2837
[syft.FloatTensor of size 5]

In [ ]:
z

#### Torch tests (not completely working so far)

In [4]:
x = torch.FloatTensor([1, 2, 3, 4])
y = torch.FloatTensor([[1, 2, 3, 4]])
z = torch.matmul(x, y.t())
assert (torch.equal(z, torch.FloatTensor([30])))

z = torch.add(x, y)
assert (torch.equal(z, torch.FloatTensor([[2, 4, 6, 8]])))

x = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]])
y = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]])
z = torch.cross(x, y, dim=1)
assert (torch.equal(z, torch.FloatTensor([[0, 0, 0], [0, 0, 0], [0, 0, 0]])))

x = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]])
y = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]])
z = torch.dist(x, y)
assert (torch.equal(torch.FloatTensor([z]), torch.FloatTensor([0])))

x = torch.FloatTensor([1, 2, 3])
y = torch.FloatTensor([1, 2, 3])
z = torch.dot(x, y)
# There is an issue with some Macs getting 0.0 instead
# Solved here: https://github.com/pytorch/pytorch/issues/5609
assert torch.equal(torch.FloatTensor([z]), torch.FloatTensor([14]))

z = torch.eq(x, y)
assert (torch.equal(z, torch.ByteTensor([1, 1, 1])))

z = torch.ge(x, y)
assert (torch.equal(z, torch.ByteTensor([1, 1, 1])))

x = torch.FloatTensor([1, 2, 3, 4, 5])
y = torch.FloatTensor([1, 2, 3, 4, 5])
assert (x.add_(y) == torch.FloatTensor([2, 4, 6, 8, 10])).all()

In [5]:
x = torch.FloatTensor([1, 2, -3, 4, 5]).send(remote)
assert (x.abs().get() == torch.FloatTensor([1, 2, 3, 4, 5])).all()

x = torch.FloatTensor([1, 2, -3, 4, 5]).send(remote)
assert (x.cos().int().get() == torch.IntTensor(
    [0, 0, 0, 0, 0])).all()
y = x.cos_()
assert (y.cos_().int().get() == torch.IntTensor(
    [0, 0, 0, 0, 0])).all()
x = torch.FloatTensor([1, 2, -3, 4, 5]).send(remote)
assert (x.ceil().get() == torch.FloatTensor([1, 2, -3, 4, 5])).all()

assert (x.cpu().get() == torch.FloatTensor([1, 2, -3, 4, 5])).all()

In [6]:
x = torch.FloatTensor([1, 2, 3, 4, 5]).send(remote)
y = torch.FloatTensor([1, 2, 3, 4, 5]).send(remote)
assert (x.add_(y).get() == torch.FloatTensor([2,4,6,8,10])).all()

x = torch.FloatTensor([1, 2, 3, 4]).send(remote)
y = torch.FloatTensor([[1, 2, 3, 4]]).send(remote)
z = torch.matmul(x, y.t())
assert (torch.equal(z.get(), torch.FloatTensor([30])))

z = torch.add(x, y)
assert (torch.equal(z.get(), torch.FloatTensor([[2, 4, 6, 8]])))

x = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]]).send(remote)
y = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]]).send(remote)
z = torch.cross(x, y, dim=1)
assert (torch.equal(z.get(), torch.FloatTensor([[0, 0, 0], [0, 0, 0], [0, 0, 0]])))

x = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]]).send(remote)
y = torch.FloatTensor([[1, 2, 3], [3, 4, 5], [5, 6, 7]]).send(remote)
z = torch.dist(x, y)
t = torch.FloatTensor([z])
assert (torch.equal(t, torch.FloatTensor([0.])))

x = torch.FloatTensor([1, 2, 3]).send(remote)
y = torch.FloatTensor([1, 2, 3]).send(remote)
z = torch.dot(x, y)
t = torch.FloatTensor([z])
assert torch.equal(t, torch.FloatTensor([14]))

z = torch.eq(x, y)
assert (torch.equal(z.get(), torch.ByteTensor([1, 1, 1])))

z = torch.ge(x, y)
assert (torch.equal(z.get(), torch.ByteTensor([1, 1, 1])))

AttributeError: 'float' object has no attribute 'id'

In [7]:
x = torch.FloatTensor([1, 2, 3]).send(remote)
y = torch.FloatTensor([1, 2, 3]).send(remote)
z = torch.FloatTensor([1, 2, 3]).send(remote)
assert (torch.equal(torch.addcmul(z,  2, x, y).get(), torch.FloatTensor([3., 10., 21.])))

x = torch.FloatTensor([1, 2, 3]).send(remote)
y = torch.FloatTensor([1, 2, 3]).send(remote)
z = torch.FloatTensor([1, 2, 3]).send(remote)
z.addcmul_(2, x, y)
assert (torch.equal(z.get(), torch.FloatTensor([3., 10., 21.])))

x = torch.FloatTensor([[1, 2]]).send(remote)
y = torch.FloatTensor([[1, 2, 3], [4, 5, 6]]).send(remote)
z = torch.FloatTensor([1, 2, 3]).send(remote)
assert (torch.equal(torch.addmm(z, x, y).get(), torch.FloatTensor([[10., 14., 18.]])))